In [1]:
# -*- coding: utf-8 -*-

In [2]:
%matplotlib inline

###Importation des modules

In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import PDM8
from PDM8 import *
import pickle
from lxml.html import builder as E
import string
from IPython.core.display import HTML,display

In [4]:
PDM8.seuilDistribution=0.00

In [5]:
with open('Regles.pkl', 'rb') as input:
    PDM8.analyse = pickle.load(input)
with open('Classes.pkl', 'rb') as input:
    PDM8.classification = pickle.load(input)

In [6]:
bdlexIn=[u"S",u"J",u"E",u"ê",u"â",u"ô",u"6"]
ipaOut=[u"ʃ",u"ɲ",u"e",u"ɛ̃",u"ɑ̃",u"ɔ̃",u"ə"]
def recoder(chaine):
    if isinstance(chaine,str):
        chaine=chaine.decode("utf8")
    for n,element in enumerate(bdlexIn):
        chaine=chaine.replace(element,ipaOut[n])
    return chaine
print recoder(u"finisâ finisô pêtâ")

finisɑ̃ finisɔ̃ pɛ̃tɑ̃


In [7]:
def html_table(table):
    htmlTable=[]
    htmlTable.append('<table border="1"><tbody>')
    for ligne in table:
        htmlTable.append('  <tr>')
        for element in ligne:
            htmlTable.append('    <td>%s</td>'%element)
        htmlTable.append('  </tr>')
    htmlTable.append('<tbody></table>')
    return "\n".join(htmlTable)
    
class ConjugaisonTableau:
    def __init__(self,nom,liste):
        self.nom=nom
        self.finies=[]
        self.nonfinies=[]
        tradPersonnesFinies={"1":"1SG","2":"2SG","3":"3SG","4":"1PL","5":"2PL","6":"3PL"}
#        personnesFinies=tradPersonnesFinies.keys()
        personnesFinies=["1","2","3","4","5","6"]
        enteteFinies=["temps"]
        for personne in personnesFinies:
            enteteFinies.append(tradPersonnesFinies[personne])
        self.finies.append(enteteFinies)
#        self.tableau.append(enteteTableau)
        tradTempsFinies={"prs":"present","ipf":"imperfective","pst":"past","fut":"future",
                     "sbjv":"present subj.","pst.sbj":"imperfective subj.",
                     "cond":"conditional",
                     "imp":"imperative"}
        tempsFinies=["prs","ipf","pst","fut","sbjv","pst.sbj","cond","imp"]
        tradNonFinies={"inf":"infinitive","prs.pcp":"present part.","pst.pcp":"past part."}
        nonFinies=["inf","prs.pcp""pst.pcp"]
#        enteteNonFinies=tradNonFinies.values()
        tempTableau=[]
        for temps in tempsFinies:
            ligne=[]
            for personne in personnesFinies:
                ligne.append("")
            tempTableau.append(ligne)
        for element in liste:
            case,forme=element.split("-")
            if case[-1] in "123456":
#                print case
                temps,personne=case.rsplit(".",1)
                numTemps=tempsFinies.index(temps)
                numPers=personnesFinies.index(personne)
                tempTableau[numTemps][numPers]=recoder(forme)
            else:
                self.nonfinies.append([case,recoder(forme)])
        for n,ligne in enumerate(tempTableau):
#            print n,ligne,[tempsFinies[n]]+ligne
            self.finies.append([tradTempsFinies[tempsFinies[n]]]+ligne)
            
            
def prettyClique(clique):
    '''
    remplir un dict tableau pour afficher un paradigme
    '''
    tableau={'NF':{}}
    for element in sorted(clique):
        case,forme=element.split("-")
        try:
            temps,personne=case.split(".")
            if personne in "123456":
                if not temps in tableau:
                    tableau[temps]={}
                tableau[temps][personne]=forme
            else:
                tableau['NF'][temps]={personne:forme}
        except:
            tableau['NF'][case]=forme
#    pretty(tableau)
        
def prettyDict(d, indent=0):
   for key, value in d.iteritems():
      print '\t' * indent + str(key)
      if isinstance(value, dict):
         prettyDict(value, indent+1)
      elif isinstance(value, list):
        for element in value:
            print '\t' * (indent+1) + (element)
        print
      else:
         print '\t' * (indent+1) + str(value)    
                
def faireCliques(lexical):
    dataCliques={}
    cliques=list(nx.algorithms.clique.find_cliques(lexical.graphe))
    dataCliques["nb"]=len(cliques)
    nMaxClique=0
    tableaux=[]
    for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
        total=0
        for node in sorted(clique):
            poids=lexical.graphe.node[node]["weight"]
            total+=poids
#        dataCliques[n]={"taille":len(clique),"représentant":clique[0],"clique":sorted(clique)}
        conjugaison=ConjugaisonTableau("test",sorted(clique))
        tableFinies=html_table(conjugaison.finies)
        tableNonFinies=html_table(conjugaison.nonfinies)
        tables=([lexeme.encode('utf8'),len(clique),total/len(clique)],tableFinies,tableNonFinies)
        tableaux.append(tables)
        if len(clique)>=nMaxClique:
            nMaxClique=len(clique)
            total=0
            for node in sorted(clique):
                poids=lexical.graphe.node[node]["weight"]
                total+=poids
#                print(node,poids)
#            print (len(clique),clique[0])
#            print (sorted(clique))
#            prettyClique(clique)
#            print (total/len(clique))
#            print ()
    return (cliques, tableaux)

In [13]:
def faireGraphes(lexical):
    lexical.calculerParadigme()
    (cliques,tableaux)=faireCliques(lexical)
#    return lexical, cliques, tableaux
    nx.write_dot(lexical.graphe,prefixeLexeme+u"graphe.dot")
#    nx.write_dot(lexical.digraphe,prefixeLexeme+u"digraphe.dot")
    for n in range(len(cliques)):
#        print (n)
        digraphe=lexical.digraphe.subgraph(cliques[n])
#        nx.write_dot(digraphe,prefixeLexeme+u"clique%d.dot"%n)
    textHTML='<html><head>'
    textHTML+='<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />'
    textHTML+='<style>'
    textHTML+='table {font-family:"Lucida Sans Unicode", "Lucida Grande", sans-serif}'
    textHTML+='</style>'
    textHTML+='</head><body>\n'
    for table in tableaux:
        textHTML+="<center><b>\n"
        for element in table[0]:
            textHTML+=str(element)+"\n"
        textHTML+="</b>\n"
        textHTML+=table[1].encode('utf8')
        textHTML+=(table[2]+"\n").encode('utf8')
        textHTML+="</center><br/><br/>\n"
    textHTML+="</body></html>"
    with open(prefixeLexeme+"Cliques.html", 'w') as output:
        output.write(textHTML)

In [14]:
lexique={
            "finir-INF":{"inf":"finir"},
            "finir-PRS3":{"prs.3":"fini"},
            "finir-PRS4":{"prs.4":"finisô"},
            "finir-PRS34":{"prs.3":"fini","prs.4":"finisô"},
            "aboyer-INF":{"inf":"abwajE"},
            "aboyer-PRS3":{"prs.3":"abwa"},
            "aboyer-PRS4":{"prs.4":"abwajô"},
            "aboyer-PRS66":{"prs.6":["abwa","abwaj"]},
            "balayer-PRS4":{"prs.4":"balEjô"},
            "cacher-IMP2":{"imp.2":"kaS"},
            "avoir-PRS1":{"prs.1":"E"},
            "suivre-PRS1":{"prs.1":"sHi"},
            "bougre-INF":{"inf":"bugr"},
            "jitre-INF":{"inf":"Zitr"},
            "clore-INF":{"inf":"klor"},
            "courir-INF":{"inf":"kurir"},
            "distraire-INF":{"inf":"distrEr"},
            "cahincaher-INF":{"inf":"kaêkaE"},
            "fiche-PSTPCP":{"pst.pcp":"fiSy"},
            "frire-PRS3":{"prs.3":"fri"},
            "courir-FUT3":{"fut.3":"kurra"},
            "battre-FUT3":{"fut.3":"batra"},
            "peindre-FUT3":{"fut.3":"pêdra"},
            "savoir-PRS2IMP2":{"prs.2":"sE","imp.2":"saS"},
            "savoir-IMP2":{"imp.2":"saS"},
         }

In [44]:
lexique={
#    "casser-IMP2":{"imp.2":"kas"},
#    "casser-PRS6":{"prs.6":"kas"},    
#    "casser-SUBJ6":{"sbjv.6":"kas"},   
    "aboyer-PRS3":{"prs.3":"abwa"},

}

In [45]:
PDM8.verbose=False
for lexeme in lexique:
    lexical=Paradigme()
    prefixeLexeme=lexeme+"-"
    print lexeme
    cases={}
    for case in lexique[lexeme]:
        cases[case]=Case(case)
        if isinstance(lexique[lexeme][case],list):
            for element in lexique[lexeme][case]:
                cases[case].addForm(FormeCoef(element.decode('utf8'),1))
        else:
            cases[case].addForm(FormeCoef(lexique[lexeme][case].decode('utf8'),1))
    for case in cases:
        lexical.addEntree(cases[case])
    faireGraphes(lexical)

aboyer-PRS3


In [ ]:
rCliques

In [ ]:
for element in lexical.graphe.nodes():
    print lexical.graphe.node[element]
#    nom=element.encode("utf8")
    print element, type(element)==str

In [24]:
import networkx as nx
#from unidecode import unidecode
G=nx.Graph()
G.add_node("hiver", label="été")
G.add_edge(u"E~tE~",u"hiver")
G.node["hiver"]
print G.nodes()

['hiver', u'E~tE~']


In [25]:
P=nx.write_dot(G,"TEST.dot")

UnicodeEncodeError: 'ascii' codec can't encode characters in position 31-32: ordinal not in range(128)